<a href="https://colab.research.google.com/github/allyssonallan/AEBigData2022/blob/main/FBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!tar -xf AmazonMusic.tar.xz
!python3 -m pip install caserecommender

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=f187cf73726405ed35b1901ca4728aec6e227527e62a267f72b789aae90aa0e6
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget

Saved under AmazonMusic.tar.xz
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 5.4 MB/s 


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset = pd.read_json('./AmazonMusic/Digital_Music_5.json', lines=True)
dataset_metadata = pd.read_csv('./AmazonMusic/amazon_music_metadata.csv')

df = dataset[['reviewerID', 'asin', 'overall']]
df = df.merge(dataset_metadata[['asin', 'title']])

map_users = {user: idx for idx, user in enumerate(df.reviewerID.unique())}
map_items = {item: idx for idx, item in enumerate(df.asin.unique())}
df['reviewerID'] = df['reviewerID'].map(map_users)
df['asin'] = df['asin'].map(map_items)

from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)

train.to_csv('train.txt', index=False, header=False, sep='\t')
test.to_csv('test.txt', index=False, header=False, sep='\t')

In [ ]:
dataset_metadata.head()

,asin,title,Accessories,Acid Jazz,Acoustic Blues,Adult Alternative,Adult Contemporary,Africa,Afro Brazilian,Afro-Cuban,...,Wall Stickers,Wall Switches,Washers,Wave Washers & Wave Springs,Wedding Music,West Coast,West Coast Blues,Western Swing,World Dance,World Music
0,5555991584,Memory of Trees,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6308051551,Dont Drink His Blood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7901622466,On Fire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B0000000ZW,Changing Faces,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B00000016W,Pet Sounds,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
new_metadata = dataset_metadata.iloc[:,1:]
new_metadata = new_metadata.melt(id_vars=["title"])
new_metadata = new_metadata[new_metadata.value != 0]
new_metadata.reset_index(inplace=True, drop=True)
new_metadata.head()

,title,variable,value
0,Light to Dark,Acid Jazz,1.0
1,Supernatural,Acid Jazz,1.0
2,Party to the Bus Stop,Acid Jazz,1.0
3,Stratosphere Breakdown,Acid Jazz,1.0
4,Mission Impossible,Acid Jazz,1.0


In [ ]:
map_title = {}
inverse_map_title = {}

for _, row in df.iterrows():
    map_title[row.asin] = row.title
    inverse_map_title[row.title] = row.asin

In [ ]:
new_metadata['asin_id'] = new_metadata['title'].map(inverse_map_title)

In [ ]:
new_metadata.dropna(inplace=True)
new_metadata = new_metadata[['asin_id', 'variable', 'value']]
new_metadata['asin_id'] = new_metadata.asin_id.astype(int)
new_metadata

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,asin_id,variable,value
6,132,Acid Jazz,1.0
8,243,Acid Jazz,1.0
11,545,Acid Jazz,1.0
12,587,Acid Jazz,1.0
13,601,Acid Jazz,1.0
...,...,...,...
62553,2541,World Music,1.0
62561,2550,World Music,1.0
62562,2555,World Music,1.0
62563,2555,World Music,1.0


In [ ]:
new_metadata.to_csv('items_metadata.dat', index=False, sep='\t', header=False)

### Case Recommender


In [ ]:
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN

In [ ]:
ItemAttributeKNN('train.txt', 'test.txt', metadata_file='items_metadata.dat', as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 5099 users and 2582 items (41436 interactions) | sparsity:: 99.69%
test data:: 3818 users and 2288 items (10360 interactions) | sparsity:: 99.88%

training_time:: 5.963499 sec
>> metadata:: 2521 items and 292 metadata (26048 interactions) | sparsity:: 96.46%
prediction_time:: 0.506548 sec
Eval:: MAE: 0.689393 RMSE: 0.976026 
